<a href="https://colab.research.google.com/github/ikram9820/Fastdl_v3-v5/blob/main/dl2%20v5/sd_dreambooth_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq diffusers==0.4.1  transformers gradio ftfy

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from tqdm.auto import tqdm

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
path = 'ikram9820/ikram-20-im'
pipe = StableDiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16 ).to("cuda")

In [ ]:
prompt = "symmetry portrait of sks person, sci - fi, tech wear, glowing lights intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha " #@param {type:"string"}
seed =3804256875 #@param {type:"number"}
generator =torch.Generator("cuda").manual_seed( seed)
gs = 13 #@param {type:"number"}
step = 50 #@param {type:"number"}

num_samples = 1 #@param {type:"number"}
num_rows = 2 #@param {type:"number"}
all_images = [] 
for _ in range(num_rows):
    images = pipe(prompt, num_images_per_prompt=num_samples,generator=generator, num_inference_steps=step, guidance_scale=gs).images
    all_images.extend(images)

grid = image_grid(all_images, num_samples, num_rows)
grid

In [ ]:
import gradio as gr

def inference(prompt, num_samples):
    all_images = [] 
    images = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=50, guidance_scale=7.5).images
    all_images.extend(images)
    return all_images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="prompt")
            samples = gr.Slider(label="Samples",value=1)
            run = gr.Button(value="Run")
        with gr.Column():
            gallery = gr.Gallery(show_label=False)

    run.click(inference, inputs=[prompt,samples], outputs=gallery)
    gr.Examples([["a photo of sks toy riding a bicycle", 1,1]], [prompt,samples], gallery, inference, cache_examples=False)


demo.launch()
